<font face = "Times New Roman">

# ***Volume per dealer notebook*** 

### ***This notebook is used for checking dealers volume split for a big order*** 


<font>

In [1]:
import pandas as pd
import numpy as np
import importlib
import datetime as dt
import requests
import sys
import os
import logging
import time

sys.path.append(os.environ["HOME"]+"/trading/python/lib")
import talos_utils
import keys_utils as keys 



In [2]:
# Talos client init

# Talos principal client init
talos_api = keys.talos_principal_api_key()
talos_secret_api = keys.talos_principal_api_secret()
host = keys.talos_principal_host()
talos = talos_utils.Talos(talos_api, talos_secret_api, host)

# Talos whitelabel client init
talos_whitelabel_api = keys.talos_whitelabel_api_key()
talos_whitelabel_secret_api = keys.talos_whitelabel_api_secret()
host_whitelabel_host = keys.talos_whitelabel_host()
talos_wl = talos_utils.Talos(talos_whitelabel_api, talos_whitelabel_secret_api, host_whitelabel_host)


/home/eddie_akers_anchorlabs_com/.local/lib/python3.9/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


<font face = "Times New Roman">


### ***Input*** 

#### 1. Order ID List 
#### 2. Cutoff time for numbers checking. This is in Easter timezone
#### 3. Talos env, "a1" or "hold"  


<font>

In [21]:
all_orders = [
'40e08590-b6de-48b3-9750-851fb80353e3',
'630996e3-b926-445c-82df-4d9417ca701d']


 # input all order ID into the list, use Microstrategy as an example here
dealer_cutoff_start = dt.datetime(2025, 3, 26, 8, 0, 0, 0) # in EST
dealer_cutoff_end = dt.datetime(2025, 6, 21, 22, 0, 0, 0)  # in EST
env = "hold"   # or "a1"``

In [22]:
def multi_order_fills(all_orders,talos_env):
    combined_df = pd.DataFrame()  # Initialize an empty dataframe to store the combined data
    for identifier in all_orders:
        # Call run_function for each identifier and get the dataframe
        try:
            df = talos_env.get_trade_fills(order_id=identifier)
            df['Amount'] = df['Amount'].astype(float)
            df['Fee'] = df['Fee'].astype(float)
            df['Quantity'] = df['Quantity'].astype(float)
            df["amount_less_fees"] = df["Amount"] - df["Fee"]
            # Concatenate the dataframe to the combined dataframe
            combined_df = pd.concat([combined_df, df], ignore_index=True)
            time.sleep(0.100)
        except Exception as e:
            err_str = f"ERROR during get_trade_fills for orderID {identifier}: {str(e)}\n\n"
            print(err_str)
    return combined_df

def fills_by_dealer(df, dealer_cutoff_start=None, dealer_cutoff_end=None):
 
    if dealer_cutoff_start and dealer_cutoff_end is not None:
        filtered_df = df.loc[(df['Timestamp'] >= dealer_cutoff_start)
                             & (df['Timestamp'] < dealer_cutoff_end)]
    else:
        filtered_df = df
    # Group by dealer
    df_by_dealer = filtered_df.groupby("Market")[["amount_less_fees", "Quantity"]].sum().reset_index()
    df_count = filtered_df.groupby("Market")["OrderID"].count().reset_index()
    df_count.rename(columns={"OrderID": "fill_count"})
    
    pd.options.display.float_format = '{:,.8f}'.format
    amount = filtered_df['amount_less_fees'].sum()
    qty = filtered_df['Quantity'].sum() 

    summary_str = f"Amount: ${amount:,.4f} \nQuantity: {qty:,.4f}\n"
    summary_df = df_by_dealer.merge(df_count)
    return filtered_df, summary_str, summary_df

In [23]:
talos_env = talos if env == "hold" else talos_wl
combined_df = multi_order_fills(all_orders,talos_env)
filtered_df, summary_str, summary_df = fills_by_dealer(combined_df, dealer_cutoff_start, dealer_cutoff_end)
print(summary_str)
summary_df = summary_df.set_index('Market')
summary_df = summary_df.rename(columns={'OrderID': 'fill_count', 'Quantity': 'quantity'})
summary_df.loc["TOTAL"] = summary_df.sum()
summary_df["win_rate"] = summary_df["quantity"] / summary_df.loc["TOTAL", "quantity"]

summary_df

Amount: $149,954,576.4492 
Quantity: 1,728.4145



,amount_less_fees,quantity,fill_count,win_rate
Market,,,,
cumberland,"21,435,486.13920575",247.19444942,"1,926.00000000",0.14301803
galaxy,"37,207,508.49369272",428.34146832,"2,584.00000000",0.24782334
janestreet,"52,102,367.29282686",601.13708114,"6,347.00000000",0.34779682
nonco,"34,534,984.89347900",397.89747200,"1,301.00000000",0.23020951
wintermute,"4,674,229.63000000",53.84406680,584.00000000,0.03115229
TOTAL,"149,954,576.44920433","1,728.41453768","12,742.00000000",1.00000000


In [24]:
summary_df.sum()

amount_less_fees   299,909,152.89840865
quantity                 3,456.82907536
fill_count              25,484.00000000
win_rate                     2.00000000
dtype: float64